<a href="https://colab.research.google.com/github/everardogarciaf/portfolio/blob/main/Everardo_Garcia_Flores_senior_data_analyst_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Senior Analyst SQL Homework

You will be working with a fictious ecommerce dataset provided by Looker, hosted publicly on BigQuery https://console.cloud.google.com/marketplace/product/bigquery-public-data/thelook-ecommerce

Click this link for a sample query against this data set
https://console.cloud.google.com/bigquery?sq=258440106949:bfb3c07d98e6487ca028cd23ea86d361


You can use the Explorer as your scratch, but will write your final queries in this notebook and save the code and the results when you are done and send the link to your saved result back to -------. 

You will write some SQL to return some specific summaries, then you will perform a free response analysis from a prompt. Spend no more than a few hours total completing this and email immediately when you're done

In [12]:
# first you must authenticate with your Google account to query against the project you set up
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
# prettier table display
%load_ext google.colab.data_table

#Query Assignment 1

Your partner in the marketing department wants a list of past customers to send a promotion. 
Fill in and run the cell below with a query that returns all user ids, first and last names and emails, who have ordered at least $20 lifetime, total over all their orders (go to the Explorer to browse the schema to find the necessary tables)

In [53]:
%%bigquery --project cypress-hw

##Query Assignment 1
##--Pull the total sale price for each order
WITH order_totals AS(
SELECT
    o.order_id
    , o.user_id
    , count(o.order_id) AS num_of_items
    , sum(oi.sale_price) AS order_total
FROM bigquery-public-data.thelook_ecommerce.orders o
LEFT JOIN bigquery-public-data.thelook_ecommerce.order_items oi
    ON oi.order_id = o.order_id
GROUP BY 1, 2
)

##--Find all users and their details, then get the sum of the total on all that user's orders
##--for users with a lifetime order total >= $20
SELECT
    u.id AS user_id
    , u.first_name
    , u.last_name
    , u.email
    , count(DISTINCT ot.order_id) AS num_of_orders
    , sum(order_total) AS lifetime_order_total
FROM bigquery-public-data.thelook_ecommerce.users u
LEFT JOIN order_totals ot
    ON ot.user_id = u.id
GROUP BY 1, 2, 3, 4
HAVING lifetime_order_total >= 20
ORDER BY 6 DESC

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,first_name,last_name,email,num_of_orders,lifetime_order_total
0,67470,Michael,Morris,michaelmorris@example.net,2,1694.910004
1,46724,David,Huang,davidhuang@example.org,4,1663.399994
2,48411,James,Holmes,jamesholmes@example.com,4,1663.299999
3,61272,David,Jackson,davidjackson@example.org,4,1588.160004
4,36367,Jeffrey,Jackson,jeffreyjackson@example.org,4,1556.700003
...,...,...,...,...,...,...
72962,39588,Erika,Yates,erikayates@example.com,1,20.000000
72963,80234,Bruce,Rivera,brucerivera@example.com,1,20.000000
72964,17344,Amy,Trujillo,amytrujillo@example.org,1,20.000000
72965,90655,Nicholas,Hunt,nicholashunt@example.org,1,20.000000


Hey Marketing Team! Be sure to send the promotion to the customers at the top of the list. They're some of our best customers! :)

------------------------------------------------------------------------------

#Query Assignment 2

Your partner in finance and operations wants to know total profit last month (Nov 2022) by distribution center, to see if any of the distributions centers are performing much better than others. Total profit is the sale price of items sold from orders (ignore unsold inventory) minus the product cost of sold items found in the inventory table (for simplicity assume everything sold at full retail price, no discounts, returns or shrink). For extra credit, provide the month over month diff in the profit margin, from Oct 2022 to Nov 2022. 

In [60]:
%%bigquery --project cypress-hw
# replace 'your-project-id' with your personal project id where you linked the data set

##Query Assignment 2
##--Find each ordered item
##--Assume the date on which the order was created is the finance order date
WITH all_ordered_items AS(
SELECT DISTINCT
    oi.id as order_item_id
    , o.order_id
    , o.user_id
    , i.product_id
    , i.product_distribution_center_id as dist_center_id
    , date_trunc(CAST(o.created_at AS DATE), MONTH) AS order_month
    , oi.sale_price AS revenue
    , i.cost
    # , oi.sale_price - i.cost AS item_profit
FROM bigquery-public-data.thelook_ecommerce.orders o
LEFT JOIN bigquery-public-data.thelook_ecommerce.order_items oi
    ON oi.order_id = o.order_id
LEFT JOIN bigquery-public-data.thelook_ecommerce.inventory_items i
    ON i.product_id = oi.product_id
)

##--Find the monthly financials by distribution center
, monthly_financials AS (
SELECT
  a.order_month
  , a.dist_center_id
  , d.name
  , sum(a.cost) AS total_cost
  , sum(a.revenue) AS total_revenue
  , count(DISTINCT a.order_id) AS number_of_orders
FROM all_ordered_items a
LEFT JOIN bigquery-public-data.thelook_ecommerce.distribution_centers d
    ON d.id = a.dist_center_id
WHERE order_month BETWEEN '2022-06-01' AND '2022-11-30'
GROUP BY 1, 2, 3
ORDER BY 2, 1, total_revenue DESC
)

##--Find the monthly financials for the previous month
, previous_month_financials AS (
SELECT
    m.dist_center_id
    , m.name AS dist_center_location
    , m.order_month
    , m.total_cost AS monthly_cost
    , m.total_revenue AS monthly_revenue
    , m.total_revenue - m.total_cost AS monthly_profit
    , (m.total_revenue - m.total_cost) / m.total_revenue * 100 AS monthly_profit_margin_pct
    , LAG(m.total_cost, 1, 0) OVER(PARTITION BY m.dist_center_id ORDER BY m.dist_center_id, m.order_month) AS previous_month_cost
    , LAG(m.total_revenue, 1, 0) OVER(PARTITION BY m.dist_center_id ORDER BY m.dist_center_id, m.order_month) AS previous_month_revenue
    , SUM(number_of_orders) AS total_orders
FROM monthly_financials m
GROUP BY 1, 2, 3, 4, 5, 6 
ORDER BY 1
)

##--Find the profit and margin for the previous month 
, previous_month_margin AS(
SELECT *
  , p.previous_month_revenue - p.previous_month_cost AS previous_month_profit
  , (p.previous_month_revenue - p.previous_month_cost) / p.previous_month_revenue * 100 AS previous_month_profit_margin_pct
FROM previous_month_financials p
WHERE order_month = '2022-11-01'
)

##--Determine the MoM Profit Margin Diff
SELECT *
  , pm.monthly_profit_margin_pct - pm.previous_month_profit_margin_pct AS month_over_month_profit_margin_pct_difference
FROM previous_month_margin pm
ORDER BY monthly_profit DESC, month_over_month_profit_margin_pct_difference DESC

Query is running:   0%|          |

Downloading:   0%|          |

,dist_center_id,dist_center_location,order_month,monthly_cost,monthly_revenue,monthly_profit,monthly_profit_margin_pct,previous_month_cost,previous_month_revenue,total_orders,previous_month_profit,previous_month_profit_margin_pct,month_over_month_profit_margin_pct_difference
0,3,Houston TX,2022-11-01,46356.279366,99086.390011,52730.110645,53.216300,39056.087851,83143.010009,1397,44086.922158,53.025410,0.190890
1,1,Memphis TN,2022-11-01,44754.329519,93506.490195,48752.160676,52.137729,36493.513516,76025.430125,1541,39531.916609,51.998281,0.139448
2,2,Chicago IL,2022-11-01,42156.081429,89014.320110,46858.238681,52.641236,37081.858755,77324.380110,1490,40242.521355,52.043768,0.597468
3,8,Mobile AL,2022-11-01,40013.877980,81951.580124,41937.702144,51.173757,34887.205033,71553.770035,1156,36666.565001,51.243373,-0.069616
4,7,Philadelphia PA,2022-11-01,36359.297463,74321.500097,37962.202635,51.078359,32000.386672,65543.070101,1087,33542.683428,51.176552,-0.098193
5,6,Port Authority of New York/New Jersey NY/NJ,2022-11-01,29656.812398,62140.340046,32483.527648,52.274461,25756.182054,53671.999993,1054,27915.817939,52.011883,0.262578
6,4,Los Angeles CA,2022-11-01,29213.729573,60045.710213,30831.980640,51.347516,27563.908179,56647.050143,1067,29083.141964,51.340965,0.006551
7,5,New Orleans LA,2022-11-01,25440.788952,54228.250077,28787.461125,53.085728,22867.550919,48560.890067,835,25693.339149,52.909531,0.176197
8,10,Savannah GA,2022-11-01,27311.069245,55455.719991,28144.650746,50.751574,22220.072692,45191.209974,779,22971.137282,50.830985,-0.079411
9,9,Charleston SC,2022-11-01,21949.412748,44762.009948,22812.597199,50.964193,19559.999711,39841.919890,1043,20281.920179,50.905981,0.058212


*   **Houston** has been the most profitable center 2 months in a row with more than double the profits of the lowest distribution center. The month over month profit margin percent difference is also the third highest of all the distribution centers.
*   Chicago's profit margins increased dramatatically even compared to the runner up, NY/NJ (.6% vs .26%, respectively). Chicago is also one of the three most profitable distribution centers.
*   Let's keep an eye on Savannah, GA, Mobile AL, and Philly. Their profit margin percent saw a month over month decrease of .08%, .07%, and .10%, respectively.
*   Luckily, Mobile and Philly have enough volume to bring in good total profits (80% and 72& of top distribution center Houston's November profits, respectively).
*   Unfortunately, Savannah doesn't look as good as it has particularly low volume (50% of the busiest center, Memphis's orders) and low total profits (only 53% of Houston's November profits).






------------------------------------------------------------------------------

#Free Response Assignment

After looking at the profit margins of distribution centers, the VP of finance is very impressed with your skills and asks you look into the data more and see if you can find any more efficiencies for the distribution centers. Using your analytical best judgement, write one or more additional queries and show what you found. This should be doable 100% with SQL, but if you know Python or R for a chart you can use it in this section (this doesn't need machine learning or advanced stats techniques at all, this is pure exploratory mining). A few ideas to get started - how overstocked or understocked are they? is inventory well allocated across centers or not? Write some comments on any recommendations from your findings

NOTE: Run the cell below to get sample output of distribution center order fulfillment and financial metrics. Check out the cells further below for analysis and comments/recommendations.

In [ ]:
%%bigquery  --project cypress-hw

##--Efficiencies to look into:
#Shipping distance (what's the avg distance between the distribution center and the user?)
#Time to ship (when is the order created vs when is the order shipped?)
#Time in inventory (how long does a distribution center hold items in inventory?)
#Order profit by warehouse (Is the monthly average order profit trending up or down for a warehouse?)


##--Fullfilment log for all ordered and fulfilled items
##It appears that items that are shipped are removed from inventory, so focusing on order fulfillment instead of inventory
WITH fulfillment_log AS (
SELECT
  i.id AS inventory_item_id
  , p.id AS product_id
  , o.order_id
  , oi.user_id
  , p.category
  , p.sku
  , p.distribution_center_id AS warehouse_id
  , p.cost
  , p.retail_price
  , p.retail_price - p.cost AS item_profit
  , CAST(oi.created_at AS DATE) AS order_date
  , CAST(oi.shipped_at AS DATE) AS shipping_date
  , date_diff(oi.shipped_at, oi.created_at, DAY) AS days_to_fulfill_order
  # , CASE WHEN oi.shipped_at IS NULL THEN IFNULL(date_diff(CURRENT_TIMESTAMP(), oi.created_at, DAY),0)
  #                               ELSE IFNULL(date_diff(oi.shipped_at, oi.created_at, DAY),0) END AS days_to_fulfill_order
  # , CAST(i.created_at AS DATE) AS enter_inventory_date
  # , CAST(i.sold_at AS DATE) AS exit_inventory_date
  # , CASE WHEN i.sold_at IS NULL THEN 0 ELSE 1 END AS is_sold
  # , CASE WHEN oi.shipped_at IS NULL THEN 0 ELSE 1 END AS is_shipped
  # , CASE WHEN i.sold_at IS NULL THEN IFNULL(date_diff(CURRENT_TIMESTAMP(), i.created_at, DAY),0)
  #                               ELSE IFNULL(date_diff(i.sold_at, i.created_at, DAY),0) END AS days_in_inventory
FROM bigquery-public-data.thelook_ecommerce.products p
LEFT JOIN bigquery-public-data.thelook_ecommerce.inventory_items i
    ON i.product_id = p.id
LEFT JOIN bigquery-public-data.thelook_ecommerce.order_items oi
    ON oi.inventory_item_id = i.id
LEFT JOIN bigquery-public-data.thelook_ecommerce.orders o
    ON o.order_id = oi.order_id
WHERE 1=1
  AND oi.order_id IS NOT NULL
  AND oi.shipped_at IS NOT NULL
  AND oi.created_at <= oi.shipped_at
  #There are items that were shipped but not sold. In a practical situation, I'd look to understand this further.
  #Given the constraints of the assignment, I'm ignoring anything without an order_id and focusing on the fulfilled orders
ORDER BY days_to_fulfill_order DESC
)

##--Monthly calendar to look at H2 2022
, calendar AS (
SELECT
  date,
  extract(month from date) as calendar_month,
  extract(year from date) as calendar_year,
from unnest(generate_date_array('2022-06-01', '2022-12-31', interval 1 month)) as date
order by date
)
##--Calculate the distance between the distribution center and the shipping address
,shipping_distances AS(
SELECT
    f.order_id
    , f.product_id
    , f.inventory_item_id
    , f.warehouse_id
    , d.name AS warehouse_location
    , ST_DISTANCE(
      ST_GEOGPOINT(d.longitude, d.latitude)
      , ST_GEOGPOINT(u.longitude,u.latitude)) /1000 shipping_distance_km
FROM fulfillment_log f
LEFT JOIN bigquery-public-data.thelook_ecommerce.distribution_centers d
    ON d.id = f.warehouse_id
LEFT JOIN bigquery-public-data.thelook_ecommerce.users u
    ON u.id = f.user_id
)

##--Gather fulfillment and financial metrics per month by distribution center
SELECT
    c.date AS calendar_month
    , i.warehouse_id
    , s.warehouse_location
    # i.product_id
    # , i.category
    # , MAX(i.days_to_fulfill_order) AS longest_days_to_fulfill #--all 3 days, so ignoring
    # , MIN(i.days_to_fulfill_order) AS shortest_days_to_fulfill #--all 0 days, so ignoring
    , AVG(i.days_to_fulfill_order) AS avg_days_to_fulfill
    , count(*) AS total_fulfilled_items
    , AVG(i.cost) AS item_cost
    , AVG(i.item_profit) AS item_profit
    , AVG(i.retail_price) AS item_value
    , SUM(i.cost) AS total_cost_of_sold_items
    , SUM(i.retail_price) AS total_value_of_sold_items
    , SUM(i.item_profit) AS total_profit_of_sold_items
    , AVG(s.shipping_distance_km) AS avg_shipping_distance
FROM calendar c
LEFT JOIN fulfillment_log i ON c.date >= i.order_date
    AND c.date < coalesce(i.shipping_date, CURRENT_DATE())  #-- need coalesce in case the item hasn't been shipped yet
#-- the above join makes sure the inventory item was created before the current date and sold after the current date, or not sold yet
#-- This is no longer really necessary since we're looking at fulfillment, but left as-is in case inventory is looked at later
LEFT JOIN shipping_distances s
    ON s.inventory_item_id = i.inventory_item_id
GROUP BY 1, 2, 3
ORDER BY 2, 1, total_fulfilled_items DESC, avg_days_to_fulfill DESC


NOTE: Run the cells below to create the data frames needed for the visualizations found further below. 

In [62]:
##DATAFRAME 1 - includes Distribution Center Location

%%bigquery df --project cypress-hw


#Efficiencies to look into:
#Shipping distance (what's the avg distance between the distribution center and the user?)
#Time to ship (when is the order created vs when is the order shipped?)
#Time in inventory (how long does a distribution center hold items in inventory?)
#Order profit by warehouse (Is the monthly average order profit trending up or down for a warehouse?)



##-----Fullfilment log for all ordered and fulfilled items
##It appears that items that are shipped are removed from inventory, so focusing on order fulfillment instead of inventory
WITH fulfillment_log AS (
SELECT
  i.id AS inventory_item_id
  , p.id AS product_id
  , o.order_id
  , oi.user_id
  , p.category
  , p.sku
  , p.distribution_center_id AS warehouse_id
  , p.cost
  , p.retail_price
  , p.retail_price - p.cost AS item_profit
  , CAST(oi.created_at AS DATE) AS order_date
  , CAST(oi.shipped_at AS DATE) AS shipping_date
  , date_diff(oi.shipped_at, oi.created_at, DAY) AS days_to_fulfill_order
  # , CASE WHEN oi.shipped_at IS NULL THEN IFNULL(date_diff(CURRENT_TIMESTAMP(), oi.created_at, DAY),0)
  #                               ELSE IFNULL(date_diff(oi.shipped_at, oi.created_at, DAY),0) END AS days_to_fulfill_order
  # , CAST(i.created_at AS DATE) AS enter_inventory_date
  # , CAST(i.sold_at AS DATE) AS exit_inventory_date
  # , CASE WHEN i.sold_at IS NULL THEN 0 ELSE 1 END AS is_sold
  # , CASE WHEN oi.shipped_at IS NULL THEN 0 ELSE 1 END AS is_shipped
  # , CASE WHEN i.sold_at IS NULL THEN IFNULL(date_diff(CURRENT_TIMESTAMP(), i.created_at, DAY),0)
  #                               ELSE IFNULL(date_diff(i.sold_at, i.created_at, DAY),0) END AS days_in_inventory
FROM bigquery-public-data.thelook_ecommerce.products p
LEFT JOIN bigquery-public-data.thelook_ecommerce.inventory_items i
    ON i.product_id = p.id
LEFT JOIN bigquery-public-data.thelook_ecommerce.order_items oi
    ON oi.inventory_item_id = i.id
LEFT JOIN bigquery-public-data.thelook_ecommerce.orders o
    ON o.order_id = oi.order_id
WHERE 1=1
  AND oi.order_id IS NOT NULL
  AND oi.shipped_at IS NOT NULL
  AND oi.created_at <= oi.shipped_at
  #There are items that were shipped but not sold. In a practical situation, I'd look to understand this further.
  #Given the constraints of the assignment, I'm ignoring anything without an order_id and focusing on the order and ship date
ORDER BY days_to_fulfill_order DESC
)

#monthly calendar to look at 2020 through EOY 2022
, calendar AS (
SELECT
  date,
  extract(month from date) as calendar_month,
  extract(year from date) as calendar_year,
from unnest(generate_date_array('2022-06-01', '2022-12-31', interval 1 month)) as date
order by date
)
,shipping_distances AS(
SELECT
    f.order_id
    , f.product_id
    , f.inventory_item_id
    , f.warehouse_id
    , d.name AS warehouse_location
    , d.longitude AS wh_long
    , d.latitude AS wh_lat
    , u.longitude AS user_long
    , u.latitude AS user_lat
    , ST_DISTANCE(
      ST_GEOGPOINT(d.longitude, d.latitude)
      , ST_GEOGPOINT(u.longitude,u.latitude)) /1000 shipping_distance_km
FROM fulfillment_log f
LEFT JOIN bigquery-public-data.thelook_ecommerce.distribution_centers d
    ON d.id = f.warehouse_id
LEFT JOIN bigquery-public-data.thelook_ecommerce.users u
    ON u.id = f.user_id
)

SELECT
    c.date AS calendar_month
    , i.warehouse_id
    , s.warehouse_location
    , s.wh_long
    , s.wh_lat
    # , s.user_long
    # , s.user_lat
    # i.product_id
    # , i.category
    # , MAX(i.days_to_fulfill_order) AS longest_days_to_fulfill #--all 3 days, so ignoring
    # , MIN(i.days_to_fulfill_order) AS shortest_days_to_fulfill #--all 0 days, so ignoring
    , AVG(i.days_to_fulfill_order) AS avg_days_to_fulfill
    , count(*) AS total_fulfilled_items
    , AVG(i.cost) AS item_cost
    , AVG(i.item_profit) AS item_profit
    , AVG(i.retail_price) AS item_value
    , SUM(i.cost) AS total_cost_of_sold_items
    , SUM(i.retail_price) AS total_value_of_sold_items
    , SUM(i.item_profit) AS total_profit_of_sold_items
    , AVG(s.shipping_distance_km) AS avg_shipping_distance
FROM calendar c
LEFT JOIN fulfillment_log i ON c.date >= i.order_date
    AND c.date < coalesce(i.shipping_date, CURRENT_DATE())  #-- need coalesce in case the item hasn't been shipped yet
#-- the above join makes sure the inventory item was created before the current date and sold after the current date, or not sold yet
LEFT JOIN shipping_distances s
    ON s.inventory_item_id = i.inventory_item_id
GROUP BY 1, 2, 3, 4, 5
ORDER BY 2, 1, total_fulfilled_items DESC, avg_days_to_fulfill DESC


Query is running:   0%|          |

Downloading:   0%|          |

In [63]:
import plotly.express as px

px.bar(
    df,
    x='calendar_month',
    y='total_profit_of_sold_items',
    color='warehouse_location',
    barmode='group'
)

As mentioned in Query Assignment 2, Chicago has clearly been one of the most profitable distribution centers. Selecting Chicago and Savannah in the chart above further highlights the disparity in total monthly profits between the two centers. 

In [21]:
import plotly.express as px

px.line(
    df,
    x='calendar_month',
    y='avg_days_to_fulfill',
    color='warehouse_location',
)

In [42]:
import plotly.express as px

px.bar(
    df,
    x='calendar_month',
    y='avg_shipping_distance',
    color='warehouse_location',
)

In [44]:
#avg shipping distance

import datetime


fig = px.scatter_mapbox(
    df[df['calendar_month'] == datetime.datetime.strptime('2022-11-01', '%Y-%m-%d').date()],  # only plot one month of data
    lat='wh_lat',
    lon='wh_long',
    color='avg_shipping_distance', # this should include a color scale based on the number of items sold
    mapbox_style='open-street-map'
)
fig.show()

In [45]:
import datetime


fig = px.scatter_mapbox(
    df[df['calendar_month'] == datetime.datetime.strptime('2022-11-01', '%Y-%m-%d').date()],  # only plot one month of data
    lat='wh_lat',
    lon='wh_long',
    color='avg_days_to_fulfill', # this should include a color scale based on the number of items sold
    mapbox_style='open-street-map'
)
fig.show()

In [46]:
##DATAFRAME 2 - includes User Location (i.e. shipping destination)


%%bigquery userdf --project cypress-hw
# replace 'your-project-id' with your personal project id where you linked the data set


#Efficiencies to look into:
#Shipping distance (what's the avg distance between the distribution center and the user?)
#Time to ship (when is the order created vs when is the order shipped?)
#Time in inventory (how long does a distribution center hold items in inventory?)
#Order profit by warehouse (Is the monthly average order profit trending up or down for a warehouse?)



##-----Fullfilment log for all ordered and fulfilled items
##It appears that items that are shipped are removed from inventory, so focusing on order fulfillment instead of inventory
WITH fulfillment_log AS (
SELECT
  i.id AS inventory_item_id
  , p.id AS product_id
  , o.order_id
  , oi.user_id
  , p.category
  , p.sku
  , p.distribution_center_id AS warehouse_id
  , p.cost
  , p.retail_price
  , p.retail_price - p.cost AS item_profit
  , CAST(oi.created_at AS DATE) AS order_date
  , CAST(oi.shipped_at AS DATE) AS shipping_date
  , date_diff(oi.shipped_at, oi.created_at, DAY) AS days_to_fulfill_order
  # , CASE WHEN oi.shipped_at IS NULL THEN IFNULL(date_diff(CURRENT_TIMESTAMP(), oi.created_at, DAY),0)
  #                               ELSE IFNULL(date_diff(oi.shipped_at, oi.created_at, DAY),0) END AS days_to_fulfill_order
  # , CAST(i.created_at AS DATE) AS enter_inventory_date
  # , CAST(i.sold_at AS DATE) AS exit_inventory_date
  # , CASE WHEN i.sold_at IS NULL THEN 0 ELSE 1 END AS is_sold
  # , CASE WHEN oi.shipped_at IS NULL THEN 0 ELSE 1 END AS is_shipped
  # , CASE WHEN i.sold_at IS NULL THEN IFNULL(date_diff(CURRENT_TIMESTAMP(), i.created_at, DAY),0)
  #                               ELSE IFNULL(date_diff(i.sold_at, i.created_at, DAY),0) END AS days_in_inventory
FROM bigquery-public-data.thelook_ecommerce.products p
LEFT JOIN bigquery-public-data.thelook_ecommerce.inventory_items i
    ON i.product_id = p.id
LEFT JOIN bigquery-public-data.thelook_ecommerce.order_items oi
    ON oi.inventory_item_id = i.id
LEFT JOIN bigquery-public-data.thelook_ecommerce.orders o
    ON o.order_id = oi.order_id
WHERE 1=1
  AND oi.order_id IS NOT NULL
  AND oi.shipped_at IS NOT NULL
  AND oi.created_at <= oi.shipped_at
  #There are items that were shipped but not sold. In a practical situation, I'd look to understand this further.
  #Given the constraints of the assignment, I'm ignoring anything without an order_id and focusing on the order and ship date
ORDER BY days_to_fulfill_order DESC
)

#monthly calendar to look at 2020 through EOY 2022
, calendar AS (
SELECT
  date,
  extract(month from date) as calendar_month,
  extract(year from date) as calendar_year,
from unnest(generate_date_array('2022-06-01', '2022-12-31', interval 1 month)) as date
order by date
)
,shipping_distances AS(
SELECT
    f.order_id
    , f.product_id
    , f.inventory_item_id
    , f.warehouse_id
    , d.name AS warehouse_location
    , d.longitude AS wh_long
    , d.latitude AS wh_lat
    , u.longitude AS user_long
    , u.latitude AS user_lat
    , ST_DISTANCE(
      ST_GEOGPOINT(d.longitude, d.latitude)
      , ST_GEOGPOINT(u.longitude,u.latitude)) /1000 shipping_distance_km
FROM fulfillment_log f
LEFT JOIN bigquery-public-data.thelook_ecommerce.distribution_centers d
    ON d.id = f.warehouse_id
LEFT JOIN bigquery-public-data.thelook_ecommerce.users u
    ON u.id = f.user_id
)

SELECT
    c.date AS calendar_month
    , i.warehouse_id
    , s.warehouse_location
    # , s.wh_long
    # , s.wh_lat
    , s.user_long
    , s.user_lat
    # i.product_id
    # , i.category
    # , MAX(i.days_to_fulfill_order) AS longest_days_to_fulfill #--all 3 days, so ignoring
    # , MIN(i.days_to_fulfill_order) AS shortest_days_to_fulfill #--all 0 days, so ignoring
    , AVG(i.days_to_fulfill_order) AS avg_days_to_fulfill
    , count(*) AS total_fulfilled_items
    , AVG(i.cost) AS item_cost
    , AVG(i.item_profit) AS item_profit
    , AVG(i.retail_price) AS item_value
    , SUM(i.cost) AS total_cost_of_sold_items
    , SUM(i.retail_price) AS total_value_of_sold_items
    , SUM(i.item_profit) AS total_profit_of_sold_items
    , AVG(s.shipping_distance_km) AS avg_shipping_distance
FROM calendar c
LEFT JOIN fulfillment_log i ON c.date >= i.order_date
    AND c.date < coalesce(i.shipping_date, CURRENT_DATE())  #-- need coalesce in case the item hasn't been shipped yet
#-- the above join makes sure the inventory item was created before the current date and sold after the current date, or not sold yet
LEFT JOIN shipping_distances s
    ON s.inventory_item_id = i.inventory_item_id
GROUP BY 1, 2, 3, 4, 5
ORDER BY 2, 1, total_fulfilled_items DESC, avg_days_to_fulfill DESC


Query is running:   0%|          |

Downloading:   0%|          |

In [49]:
import datetime


fig = px.density_mapbox(
    userdf[userdf['calendar_month'] == datetime.datetime.strptime('2022-11-01', '%Y-%m-%d').date()],  # only plot one month of data
    lat='user_lat',
    lon='user_long',
    z='avg_days_to_fulfill', # this should include a color scale based on the number of items sold
    mapbox_style='open-street-map'
)
fig.show()